In [1]:
import numpy as np
import pandas as pd
import pymysql
import json

In [2]:
def get_mysql_credentials():
        with open("mysql_config.json", "r") as jsonfile:
            data = json.load(jsonfile)
        return (data['host'], data['user'], data['password'], data['database'])
    
def get_cursor():
    creds = get_mysql_credentials()
    dbinstance_endpoint = creds[0]
    db_username = creds[1]
    db_pw = creds[2]
    db_name = creds[3]
    connection = pymysql.connect(host = dbinstance_endpoint, user = db_username, password = db_pw, database = db_name, autocommit=True)
    cursor = connection.cursor()
    return cursor

cursor = get_cursor()

In [38]:
import ast
from datetime import datetime, timedelta, date

In [4]:
import bisect

In [52]:
from matplotlib import pyplot as plt

In [18]:
def calc_orderboook_imb(bids, asks, level=1):
    '''
    bids: ascending [price, qty] list
    asks: ascending [price, qty] list
    level: depth of orderbook for which to calculate order book imbalance
    '''
    bids_depth = len(bids)
    asks_depth = len(asks)
    if level > min([bids_depth, asks_depth]) or level == 0:
        if level <= 1:
            if bids_depth:
                return 1
            elif asks_depth:
                return -1
            else:
                return 0
        return calc_orderboook_imb(bids, asks, min([bids_depth, asks_depth]))
    else:
        bids_vol = 0
        asks_vol = 0
        for idx in range(level):
            bids_vol += bids[len(bids)-1-idx][1]
            asks_vol += asks[idx][1]
        return (bids_vol - asks_vol)/(bids_vol + asks_vol)

In [6]:
def get_best_bid(bids):
    '''
    bids: ascending [price, qty] list
    '''
    if len(bids):
        return bids[len(bids)-1][0]
    else:
        return 0

def get_best_ask(asks):
    '''
    asks: ascending [price, qty] list
    '''
    if len(asks):
        return asks[0][0]
    else:
        return np.inf

    

In [7]:
def update_bids(bids, price, delta):
    '''
    bids: ascending [price, qty] list
    price: price level at which delta qty is being added to or removed
    delta: contracts being placed or removed from a price level
    '''
    bisect_left_ip = bisect.bisect_left(bids, price, key = lambda x: x[0])
    if bisect_left_ip == len(bids):
        #price is greater than all of the bids
        assert delta > 0
        bids.insert(bisect_left_ip, [price, delta])
        
    else:
        #price is greater than bids[:bisect_left_ip]
        if bids[bisect_left_ip][0] == price:
            #price exists in the current bids
            bids[bisect_left_ip][1]+=delta #update qty
            if bids[bisect_left_ip][1] <= 0:
                #remove price leveel
                del bids[bisect_left_ip]
        else:
            #price does not exist in the current bids
            assert delta > 0
            bids.insert(bisect_left_ip, [price, delta])

def update_asks(asks, price, delta):
    '''
    asks: ascending [price, qty] list
    price: price level at which delta qty is being added to or removed
    delta: contracts being placed or removed from a price level
    '''
    bisect_left_ip = bisect.bisect_left(asks, price, key = lambda x: x[0])
    if bisect_left_ip == len(asks):
        #price is greater than all of the asks
        assert delta > 0
        asks.insert(bisect_left_ip, [price, delta])
        
    else:
        #price is greater than asks[:bisect_left_ip]
        if asks[bisect_left_ip][0] == price:
            #price exists in the current bids
            asks[bisect_left_ip][1]+=delta #update qty
            if asks[bisect_left_ip][1] <= 0:
                #remove price leveel
                del asks[bisect_left_ip]
        else:
            #price does not exist in the current bids
            assert delta > 0
            asks.insert(bisect_left_ip, [price, delta])
    
            
    

In [39]:
date.today()

datetime.date(2024, 1, 30)

In [47]:
'''

Loops through 2024 dates and queries all of the orderbook delta messages (for all NDX and SPX range markets on Kalshi) occuring on that date that 
I have stored in a MySQL DB. Loops through each orderbook delta on a given day and updates a local state of 
the limit orderbook for that market. Appends to a dataframe, the best bid, ask, orderbook imbalance, and midprice after each delta is processed.

'''


bids = {}
asks = {}
data_dict = {'best_bid': [], 'best_ask': [], 'obi_l1': [], 'obi_l2': [], 'obi_l3': [], 'obi_l4': [], 'ticker': [], 'timestamp': []}
date_ = date(2024, 1, 2)

while date_ <= date.today():
    
    #find number of commands
    cursor.execute('select count(distinct(command_id)) from ob_snapshot where processed_ts like \'%{}%\''.format(date_.isoformat()))
    rows = cursor.fetchall()
    num_cmd_ids = None
    
    if rows is not None and len(rows) > 0:
        num_cmd_ids = rows[0][0]
    if num_cmd_ids is None:
        continue
    
    for cmd_id in range(1,num_cmd_ids+1):
        cursor.execute('select * from ob_snapshot where command_id = {} and processed_ts like \'%{}%\' order by processed_ts asc'.format(cmd_id, date_.isoformat()))
        snapshot_rows = cursor.fetchall()
        cursor.execute('select max(seq_num) from ob_snapshot where command_id = {} and processed_ts like \'%{}%\''.format(cmd_id, date_.isoformat()))
        max_seqnum = cursor.fetchall()[0][0]
        
        if max_seqnum != len(snapshot_rows):
            continue
            
        for row in snapshot_rows:
            t = row[2]
            try:
                bids_dict = ast.literal_eval(row[3])
                asks_dict = ast.literal_eval(row[4])
            except:
                continue
            bids_list = []
            asks_list = []
            for p, q in bids_dict.items():
                bids_list.append([p, q])
            for p, q in asks_dict.items():
                asks_list.append([p, q])
            
            bids_list.sort(key = lambda x:x[0])
            asks_list.sort(key = lambda x:x[0])
            
            bids[t] = bids_list[:]
            asks[t] = asks_list[:]
            
            data_dict['best_bid'].append(get_best_bid(bids[t]))
            data_dict['best_ask'].append(get_best_ask(asks[t]))
            data_dict['obi_l1'].append(calc_orderboook_imb(bids[t], asks[t], 1))
            data_dict['obi_l2'].append(calc_orderboook_imb(bids[t], asks[t], 2))
            data_dict['obi_l3'].append(calc_orderboook_imb(bids[t], asks[t], 3))
            data_dict['obi_l4'].append(calc_orderboook_imb(bids[t], asks[t], 4))
            data_dict['ticker'].append(t)
            data_dict['timestamp'].append(datetime.strptime(row[5], '%Y-%m-%dT%H:%M:%S.%fZ'))
        
        cursor.execute('select * from ob_delta where command_id = {} and processed_ts like \'%{}%\' order by processed_ts asc'.format(cmd_id, date_.isoformat()))
        delta_rows = cursor.fetchall()
        for row in delta_rows:
            t = row[2]
            price = row[3]
            delta = row[4]
            side = row[5]
            
            if side == 'yes':
                #bid
                if t in bids:
                    update_bids(bids[t], price, delta)
                else:
                    continue
            else:
                #ask
                if t in asks:
                    update_asks(asks[t], price, delta)
                else:
                    continue
            
            
            data_dict['best_bid'].append(get_best_bid(bids[t]))
            data_dict['best_ask'].append(get_best_ask(asks[t]))
            data_dict['obi_l1'].append(calc_orderboook_imb(bids[t], asks[t], 1))
            data_dict['obi_l2'].append(calc_orderboook_imb(bids[t], asks[t], 2))
            data_dict['obi_l3'].append(calc_orderboook_imb(bids[t], asks[t], 3))
            data_dict['obi_l4'].append(calc_orderboook_imb(bids[t], asks[t], 4))
            data_dict['ticker'].append(t)
            data_dict['timestamp'].append(datetime.strptime(row[6], '%Y-%m-%dT%H:%M:%S.%fZ'))
    
    date_+=timedelta(days=1)      
            

data = pd.DataFrame(data_dict)

In [49]:
data['midprice'] = data.apply(lambda x: (x['best_bid'] + x['best_ask'])/2, axis = 1)

In [51]:
data

,best_bid,best_ask,obi_l1,obi_l2,obi_l3,obi_l4,ticker,timestamp,midprice
0,0,2.0,-1.000000,-1.000000,-1.000000,-1.000000,INXD-24JAN03-B4662,2024-01-03 20:15:48.494189,1.0
1,11,13.0,-0.733333,-0.691057,-0.668712,-0.644670,INXD-24JAN03-B4687,2024-01-03 20:15:48.506276,12.0
2,72,75.0,-0.739130,-0.600000,-0.516129,-0.680136,INXD-24JAN03-B4712,2024-01-03 20:15:48.513190,73.5
3,11,12.0,-0.652174,-0.555950,-0.291856,-0.041989,INXD-24JAN03-B4737,2024-01-03 20:15:48.523760,11.5
4,0,1.0,-1.000000,-1.000000,-1.000000,-1.000000,INXD-24JAN03-B4762,2024-01-03 20:15:48.537417,0.5
...,...,...,...,...,...,...,...,...,...
630329,42,52.0,0.200000,0.250000,0.166667,0.125000,INX-24JAN30-B4912,2024-01-30 20:58:27.113487,47.0
630330,42,52.0,0.200000,0.250000,0.166667,0.125000,INX-24JAN30-B4912,2024-01-30 20:58:27.113525,47.0
630331,42,52.0,0.200000,0.250000,0.166667,0.125000,INX-24JAN30-B4912,2024-01-30 20:58:27.113549,47.0
630332,44,56.0,0.904762,0.023256,0.393258,0.207101,INX-24JAN30-B4937,2024-01-30 20:58:27.113571,50.0


In [61]:
tickers = list(data['ticker'].unique()) #unique tickers in data df
gpo = data.groupby('ticker') #groupby object of data df on ticker


In [67]:
'''
After specifying a lookahead period, this code, for each ticker, finds the correlation betweeen L1, 2, and 3
orderbook imbalance values and the midprice return after lookahead period delta messages related to the ticker.
The data is stored in a dataframe
'''

lookahead_period = 5
corr_dict = {'ticker': [], 'l1_corr': [], 'l2_corr': [], 'l3_corr': []}
for ticker in tickers:
    group = gpo.get_group(ticker)
    group = group.reset_index(drop=True)
    
    if group.shape[0] < 200:
        continue
    
    
    midprice = group['midprice'].tolist()
    obi_l1 = group['obi_l1'].tolist()[:len(midprice)-lookahead_period]
    obi_l2 = group['obi_l2'].tolist()[:len(midprice)-lookahead_period]
    obi_l3 = group['obi_l3'].tolist()[:len(midprice)-lookahead_period]
    midprice_return = (np.array(midprice[lookahead_period:])-np.array(midprice[:len(midprice)-lookahead_period]))/np.array(midprice[:len(midprice)-lookahead_period])
    
    l1_corr = np.corrcoef(obi_l1, midprice_return)[0,1]
    l2_corr = np.corrcoef(obi_l2, midprice_return)[0,1]
    l3_corr = np.corrcoef(obi_l3, midprice_return)[0,1]
    
    corr_dict['ticker'].append(ticker)
    corr_dict['l1_corr'].append(l1_corr)
    corr_dict['l2_corr'].append(l2_corr)
    corr_dict['l3_corr'].append(l3_corr)
    
corr_df = pd.DataFrame(data=corr_dict)
    


/var/folders/8r/fr9y19j57554vvz5607f_l4h0000gn/T/ipykernel_94423/3645188170.py:21: RuntimeWarning: invalid value encountered in subtract
  midprice_return = (np.array(midprice[lookahead_period:])-np.array(midprice[:len(midprice)-lookahead_period]))/np.array(midprice[:len(midprice)-lookahead_period])
/var/folders/8r/fr9y19j57554vvz5607f_l4h0000gn/T/ipykernel_94423/3645188170.py:21: RuntimeWarning: invalid value encountered in divide
  midprice_return = (np.array(midprice[lookahead_period:])-np.array(midprice[:len(midprice)-lookahead_period]))/np.array(midprice[:len(midprice)-lookahead_period])
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2698: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/numpy/lib/func

In [68]:
corr_df.describe()

,l1_corr,l2_corr,l3_corr
count,52.000000,52.000000,52.000000
mean,0.037229,0.025932,0.016916
std,0.103277,0.102333,0.100489
min,-0.324546,-0.341407,-0.341407
25%,-0.017649,-0.036229,-0.048125
50%,0.055842,0.048137,0.039039
75%,0.107518,0.088222,0.085594
max,0.209516,0.190362,0.187690


hard to claim that orderbook imbalance values are correlated with future midprice returns at all.. data points to independence

In [71]:
'''
After specifying a lookahead period, this code, for each ticker, finds the correlation betweeen L1, 2, and 3
orderbook imbalance values that < -0.75 or > 0.75 and the midprice return after lookahead period delta messages related to the ticker.
The data is stored in a dataframe
'''

lookahead_period = 5
corr_dict = {'ticker': [], 'l1_corr': [], 'l2_corr': [], 'l3_corr': []}
for ticker in tickers:
    group = gpo.get_group(ticker)
    group = group.reset_index(drop=True)
    
    if group.shape[0] < 200:
        continue
    
    
    midprice = group['midprice'].tolist()
    obi_l1 = group['obi_l1'].tolist()[:len(midprice)-lookahead_period]
    obi_l2 = group['obi_l2'].tolist()[:len(midprice)-lookahead_period]
    obi_l3 = group['obi_l3'].tolist()[:len(midprice)-lookahead_period]
    midprice_return = (np.array(midprice[lookahead_period:])-np.array(midprice[:len(midprice)-lookahead_period]))/np.array(midprice[:len(midprice)-lookahead_period])
    
    l1_x = []
    l1_y = []
    l2_x = []
    l2_y = []
    l3_x = []
    l3_y = []
    
    for idx in range(len(obi_l1)):
        if abs(obi_l1[idx]) > 0.75:
            l1_x.append(obi_l1[idx])
            l1_y.append(midprice_return[idx])
    
    for idx in range(len(obi_l2)):
        if abs(obi_l2[idx]) > 0.75:
            l2_x.append(obi_l2[idx])
            l2_y.append(midprice_return[idx])
    
    for idx in range(len(obi_l3)):
        if abs(obi_l3[idx]) > 0.75:
            l3_x.append(obi_l3[idx])
            l3_y.append(midprice_return[idx])
    
    l1_corr = np.corrcoef(l1_x, l1_y)[0,1]
    l2_corr = np.corrcoef(l2_x, l2_y)[0,1]
    l3_corr = np.corrcoef(l3_x, l3_y)[0,1]
    
    corr_dict['ticker'].append(ticker)
    corr_dict['l1_corr'].append(l1_corr)
    corr_dict['l2_corr'].append(l2_corr)
    corr_dict['l3_corr'].append(l3_corr)
    
corr_df = pd.DataFrame(data=corr_dict)
    


/var/folders/8r/fr9y19j57554vvz5607f_l4h0000gn/T/ipykernel_94423/3030747027.py:21: RuntimeWarning: invalid value encountered in subtract
  midprice_return = (np.array(midprice[lookahead_period:])-np.array(midprice[:len(midprice)-lookahead_period]))/np.array(midprice[:len(midprice)-lookahead_period])
/var/folders/8r/fr9y19j57554vvz5607f_l4h0000gn/T/ipykernel_94423/3030747027.py:21: RuntimeWarning: invalid value encountered in divide
  midprice_return = (np.array(midprice[lookahead_period:])-np.array(midprice[:len(midprice)-lookahead_period]))/np.array(midprice[:len(midprice)-lookahead_period])
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2698: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/numpy/lib/func

In [73]:
corr_df.describe()

,l1_corr,l2_corr,l3_corr
count,49.000000,49.000000,47.000000
mean,0.050147,0.062309,0.011207
std,0.169264,0.194475,0.240677
min,-0.750974,-0.595874,-0.986939
25%,-0.019880,-0.025418,-0.062444
50%,0.065918,0.041561,0.007696
75%,0.126497,0.138768,0.078835
max,0.292795,0.578149,0.817383
